Assuming that this works. We will have to test it and compare against already done files to be sure.
next steps once we confirm it works:
- batch mode: loop through many folders automatically
- auto-detect channel from folder name (if it contains “red” or “green”)
- auto-segmentation of brain ROI (no manual drawing)

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from matplotlib.widgets import PolygonSelector
from matplotlib.path import Path

In [ ]:
# ---------- Load image sequence ----------
def load_image_sequence(folder, extensions=("png", "jpg", "jpeg", "tif", "tiff")):
    files = []
    for ext in extensions:
        files += glob.glob(os.path.join(folder, f"*.{ext}"))
    files = sorted(files)

    if not files:
        raise FileNotFoundError(f"No images found in {folder}")

    stack = []
    for f in files:
        img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        stack.append(img)

    stack = np.stack(stack, axis=0)  # shape: (num_slices, H, W)
    return stack, files

In [ ]:
# ---------- Max intensity projection ----------
def max_intensity_projection(stack):
    return stack.max(axis=0)

In [ ]:
# ---------- ROI drawing with matplotlib ----------
class ROICollector:
    def __init__(self, image):
        self.image = image
        self.rois = []

        self.fig, self.ax = plt.subplots()
        self.ax.imshow(image, cmap="gray")
        self.ax.set_title("Draw ROI polygon → close shape → press ENTER to accept")
        self.selector = PolygonSelector(self.ax, self.on_select)

        self.fig.canvas.mpl_connect("key_press_event", self.on_key)

    def on_select(self, verts):
        self.verts = verts

    def on_key(self, event):
        # Press Enter to accept ROI
        if event.key == "enter":
            if hasattr(self, "verts") and len(self.verts) > 2:
                roi_name = input("Enter ROI name (e.g., brain, region1, region2): ").strip()
                self.rois.append((roi_name, self.verts))
                print(f"✅ Saved ROI: {roi_name} ({len(self.verts)} points)")
                self.ax.plot(*zip(*self.verts), linewidth=2)
                self.fig.canvas.draw()

        # Press q to quit
        if event.key == "q":
            plt.close(self.fig)

    def collect(self):
        plt.show()
        return self.rois

In [ ]:
# ---------- Convert polygon verts → mask ----------
def polygon_to_mask(image_shape, verts):
    h, w = image_shape
    y_grid, x_grid = np.mgrid[:h, :w]
    points = np.vstack((x_grid.ravel(), y_grid.ravel())).T

    poly_path = Path(verts)
    mask = poly_path.contains_points(points).reshape(h, w)
    return mask

In [ ]:
# ---------- Measure stats inside ROI ----------
def measure_roi(image, mask):
    roi_pixels = image[mask]
    return {
        "area_px": int(mask.sum()),
        "mean": float(np.mean(roi_pixels)),
        "min": float(np.min(roi_pixels)),
        "max": float(np.max(roi_pixels)),
    }

In [ ]:
# ---------- Main ----------
def run(folder, channel_name="unknown", output_csv="results.csv"):
    stack, files = load_image_sequence(folder)
    proj = max_intensity_projection(stack)

    print(f"Loaded {len(files)} images")
    print(f"Projection shape: {proj.shape}")

    # collect ROI(s)
    collector = ROICollector(proj)
    rois = collector.collect()

    rows = []
    for roi_name, verts in rois:
        mask = polygon_to_mask(proj.shape, verts)
        stats = measure_roi(proj, mask)

        row = {
            "folder": folder,
            "channel": channel_name,
            "roi_name": roi_name,
            **stats
        }
        rows.append(row)

    df_new = pd.DataFrame(rows)
    print("\nNew results:")
    print(df_new)

    # append to CSV
    if os.path.exists(output_csv):
        df_old = pd.read_csv(output_csv)
        df_all = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_all = df_new

    df_all.to_csv(output_csv, index=False)
    print(f"\nSaved/updated results in {output_csv}")

if __name__ == "__main__":
    # CHANGE THIS to your image folder path
    folder_path = r"C:\path\to\your\image_sequence_folder"
    run(folder_path, channel_name="green", output_csv="results.csv")
